In [22]:
from sqlalchemy import create_engine, text
import pandas as pd

# Database connection setup
DATABASE_TYPE = 'mysql'
DBAPI = 'mysqlconnector'
HOST = 'fashionretail.c3q4cw8i0vbc.us-east-2.rds.amazonaws.com'
USER = 'admin'
PASSWORD = 'bsan-6080-project'
DATABASE = 'fashion_retail_sales'
DATABASE_URL = f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{HOST}/{DATABASE}"
engine = create_engine(DATABASE_URL, echo=True)

# Load the CSV file
csv_file_path = 'Fashion_Retail_Sales_Clean.csv'
df = pd.read_csv(csv_file_path)

# Convert 'Date Purchase' to datetime and 'Purchase Amount (USD)' to numeric
df['DatePurchased'] = pd.to_datetime(df['Date Purchase'])
df['Revenue'] = pd.to_numeric(df['Purchase Amount (USD)'], errors='coerce')

# Remove duplicates based on 'Date Purchase', keeping the entry with the highest 'Purchase Amount (USD)'
df = df.sort_values('Revenue', ascending=False).drop_duplicates(subset='DatePurchased')

# Extract date parts
df['Day'] = df['DatePurchased'].dt.day
df['Month'] = df['DatePurchased'].dt.month
df['Quarter'] = df['DatePurchased'].dt.quarter
df['Year'] = df['DatePurchased'].dt.year

# SQL for creating tables
create_table_statements = [
    """
    CREATE TABLE IF NOT EXISTS fact_purchases (
        PurchaseID INT AUTO_INCREMENT PRIMARY KEY,
        DatePurchased DATE,
        ItemsPurchased VARCHAR(255),
        Revenue DECIMAL(10, 2)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS dim_date (
        DatePurchased DATE PRIMARY KEY,
        Day INT,
        Month INT,
        Quarter INT,
        Year INT
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS fact_customer_purchases (
        CustomerID INT AUTO_INCREMENT PRIMARY KEY,
        DatePurchased DATE,
        ItemsPurchased VARCHAR(255),
        Revenue DECIMAL(10, 2),
        PRIMARY KEY (CustomerID, DatePurchased),
        FOREIGN KEY (DatePurchased) REFERENCES dim_date(DatePurchased)
    );
    """
]

# Execute the table creation and data insertion within a single transaction
with engine.begin() as conn:
    for statement in create_table_statements:
        conn.execute(text(statement))
    
    # Insert data into dim_date
    for _, row in df[['DatePurchased', 'Day', 'Month', 'Quarter', 'Year']].drop_duplicates().iterrows():
        # Convert DatePurchased to string in the format MySQL expects
        date_purchased_str = row['DatePurchased'].date().isoformat()
        # Check if the date already exists in the table
        if not conn.execute(text("SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = :dp)"),
                            {'dp': date_purchased_str}).scalar():
            conn.execute(text("INSERT INTO dim_date (`DatePurchased`, `Day`, `Month`, `Quarter`, `Year`) VALUES (:dp, :d, :m, :q, :y)"),
                         {'dp': date_purchased_str, 'd': row['Day'], 'm': row['Month'], 'q': row['Quarter'], 'y': row['Year']})

    # Insert data into fact_purchases
    fact_purchases_df = df[['DatePurchased', 'Item Purchased', 'Revenue']].rename(columns={'Item Purchased': 'ItemsPurchased'})
    fact_purchases_df.to_sql('fact_purchases', con=engine, if_exists='append', index=False, method='multi')

    # Insert data into fact_customer_purchases
    fact_customer_purchases_df = df[['Customer Reference ID', 'DatePurchased', 'Item Purchased', 'Revenue']].rename(columns={'Customer Reference ID': 'CustomerID', 'Item Purchased': 'ItemsPurchased'})
    fact_customer_purchases_df.to_sql('fact_customer_purchases', con=engine, if_exists='append', index=False)

print("All tables populated successfully.")




2024-04-23 14:40:27,005 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-04-23 14:40:27,008 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-23 14:40:27,497 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-04-23 14:40:27,499 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-23 14:40:27,742 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-04-23 14:40:27,745 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-23 14:40:27,912 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-23 14:40:28,071 INFO sqlalchemy.engine.Engine 
    CREATE TABLE IF NOT EXISTS fact_purchases (
        PurchaseID INT AUTO_INCREMENT PRIMARY KEY,
        DatePurchased DATE,
        ItemsPurchased VARCHAR(255),
        Revenue DECIMAL(10, 2)
    );
    
2024-04-23 14:40:28,073 INFO sqlalchemy.engine.Engine [generated in 0.15923s] {}
2024-04-23 14:40:28,322 INFO sqlalchemy.engine.Engine 
    CREATE TABLE IF NOT EXISTS dim_date (
        DatePurchased DATE PRIMARY KEY,
        Day INT,
 

2024-04-23 14:40:36,776 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:40:36,778 INFO sqlalchemy.engine.Engine [cached since 8.105s ago] {'dp': '2023-07-08'}
2024-04-23 14:40:37,027 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:40:37,029 INFO sqlalchemy.engine.Engine [cached since 8.356s ago] {'dp': '2023-07-11'}
2024-04-23 14:40:37,274 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:40:37,276 INFO sqlalchemy.engine.Engine [cached since 8.603s ago] {'dp': '2022-11-10'}
2024-04-23 14:40:37,529 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:40:37,532 INFO sqlalchemy.engine.Engine [cached since 8.859s ago] {'dp': '2023-04-10'}
2024-04-23 14:40:37,780 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased =

2024-04-23 14:40:46,114 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:40:46,116 INFO sqlalchemy.engine.Engine [cached since 17.44s ago] {'dp': '2023-09-16'}
2024-04-23 14:40:46,371 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:40:46,372 INFO sqlalchemy.engine.Engine [cached since 17.7s ago] {'dp': '2023-01-28'}
2024-04-23 14:40:46,612 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:40:46,614 INFO sqlalchemy.engine.Engine [cached since 17.94s ago] {'dp': '2022-12-02'}
2024-04-23 14:40:46,860 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:40:46,862 INFO sqlalchemy.engine.Engine [cached since 18.19s ago] {'dp': '2023-03-07'}
2024-04-23 14:40:47,123 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = 

2024-04-23 14:40:55,446 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:40:55,449 INFO sqlalchemy.engine.Engine [cached since 26.78s ago] {'dp': '2023-07-03'}
2024-04-23 14:40:55,692 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:40:55,695 INFO sqlalchemy.engine.Engine [cached since 27.02s ago] {'dp': '2022-10-21'}
2024-04-23 14:40:55,940 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:40:55,943 INFO sqlalchemy.engine.Engine [cached since 27.27s ago] {'dp': '2022-11-06'}
2024-04-23 14:40:56,202 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:40:56,204 INFO sqlalchemy.engine.Engine [cached since 27.53s ago] {'dp': '2023-02-02'}
2024-04-23 14:40:56,447 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased =

2024-04-23 14:41:04,903 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:41:04,905 INFO sqlalchemy.engine.Engine [cached since 36.23s ago] {'dp': '2023-07-01'}
2024-04-23 14:41:05,158 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:41:05,160 INFO sqlalchemy.engine.Engine [cached since 36.49s ago] {'dp': '2023-09-12'}
2024-04-23 14:41:05,428 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:41:05,430 INFO sqlalchemy.engine.Engine [cached since 36.76s ago] {'dp': '2023-03-04'}
2024-04-23 14:41:05,701 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:41:05,704 INFO sqlalchemy.engine.Engine [cached since 37.03s ago] {'dp': '2022-11-15'}
2024-04-23 14:41:05,949 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased =

...

2024-04-23 14:41:43,283 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:41:43,285 INFO sqlalchemy.engine.Engine [cached since 74.62s ago] {'dp': '2023-05-21'}
2024-04-23 14:41:43,532 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:41:43,535 INFO sqlalchemy.engine.Engine [cached since 74.86s ago] {'dp': '2022-10-29'}
2024-04-23 14:41:43,781 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:41:43,783 INFO sqlalchemy.engine.Engine [cached since 75.11s ago] {'dp': '2023-04-26'}
2024-04-23 14:41:44,033 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:41:44,036 INFO sqlalchemy.engine.Engine [cached since 75.37s ago] {'dp': '2023-05-20'}
2024-04-23 14:41:44,275 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased =

2024-04-23 14:41:52,515 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:41:52,517 INFO sqlalchemy.engine.Engine [cached since 83.85s ago] {'dp': '2023-04-13'}
2024-04-23 14:41:52,764 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:41:52,767 INFO sqlalchemy.engine.Engine [cached since 84.1s ago] {'dp': '2022-10-06'}
2024-04-23 14:41:53,019 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:41:53,022 INFO sqlalchemy.engine.Engine [cached since 84.35s ago] {'dp': '2022-11-26'}
2024-04-23 14:41:53,267 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:41:53,269 INFO sqlalchemy.engine.Engine [cached since 84.6s ago] {'dp': '2023-01-30'}
2024-04-23 14:41:53,515 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %

2024-04-23 14:42:01,912 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:42:01,915 INFO sqlalchemy.engine.Engine [cached since 93.25s ago] {'dp': '2023-02-25'}
2024-04-23 14:42:02,164 INFO sqlalchemy.engine.Engine SELECT EXISTS(SELECT 1 FROM dim_date WHERE DatePurchased = %(dp)s)
2024-04-23 14:42:02,166 INFO sqlalchemy.engine.Engine [cached since 93.5s ago] {'dp': '2023-08-21'}
2024-04-23 14:42:04,303 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2024-04-23 14:42:04,306 INFO sqlalchemy.engine.Engine [generated in 0.16571s] {'table_schema': 'fashion_retail_sales', 'table_name': 'fact_purchases'}
2024-04-23 14:42:04,725 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-23 14:42:04,934 INFO sqlalchemy.engine.Engine INSERT INTO fact_purchases (`DatePurchased`, `ItemsPurchased`, `Revenue`) VALUES (%(DatePurchased_m0)s, %(Item

2024-04-23 14:42:04,937 INFO sqlalchemy.engine.Engine [no key 0.17049s] {'DatePurchased_m0': datetime.datetime(2022, 12, 20, 0, 0), 'ItemsPurchased_m0': 'Flip-Flops', 'Revenue_m0': 4932.0, 'DatePurchased_m1': datetime.datetime(2023, 8, 25, 0, 0), 'ItemsPurchased_m1': 'Shorts', 'Revenue_m1': 4872.0, 'DatePurchased_m2': datetime.datetime(2022, 12, 19, 0, 0), 'ItemsPurchased_m2': 'Sweater', 'Revenue_m2': 4859.0, 'DatePurchased_m3': datetime.datetime(2023, 5, 19, 0, 0), 'ItemsPurchased_m3': 'Jeans', 'Revenue_m3': 4771.0, 'DatePurchased_m4': datetime.datetime(2023, 4, 12, 0, 0), 'ItemsPurchased_m4': 'Tunic', 'Revenue_m4': 4661.0, 'DatePurchased_m5': datetime.datetime(2023, 2, 5, 0, 0), 'ItemsPurchased_m5': 'Handbag', 'Revenue_m5': 4619.0, 'DatePurchased_m6': datetime.datetime(2022, 11, 21, 0, 0), 'ItemsPurchased_m6': 'Romper', 'Revenue_m6': 4465.0, 'DatePurchased_m7': datetime.datetime(2022, 12, 23, 0, 0), 'ItemsPurchased_m7': 'Tunic', 'Revenue_m7': 4413.0, 'DatePurchased_m8': datetime.date

2024-04-23 14:42:05,057 INFO sqlalchemy.engine.Engine COMMIT
2024-04-23 14:42:05,477 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2024-04-23 14:42:05,479 INFO sqlalchemy.engine.Engine [cached since 1.339s ago] {'table_schema': 'fashion_retail_sales', 'table_name': 'fact_customer_purchases'}
2024-04-23 14:42:05,658 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-23 14:42:05,829 INFO sqlalchemy.engine.Engine INSERT INTO fact_customer_purchases (`CustomerID`, `DatePurchased`, `ItemsPurchased`, `Revenue`) VALUES (%(CustomerID)s, %(DatePurchased)s, %(ItemsPurchased)s, %(Revenue)s)
2024-04-23 14:42:05,831 INFO sqlalchemy.engine.Engine [generated in 0.16816s] ({'CustomerID': 4109, 'DatePurchased': datetime.datetime(2022, 12, 20, 0, 0), 'ItemsPurchased': 'Flip-Flops', 'Revenue': 4932.0}, {'CustomerID': 4040, 'DatePurchased': datetime.datetime(2023, 8, 25, 0, 0), 'ItemsPurchased': 'Short